In [1]:
%%bash
# remove everything except this notebook
shopt -s extglob
rm -rf !("notebook.ipynb") 
# to test any error in the terminal, use:
# ipython3 -c "%run notebook.ipynb"

In [12]:
!conda env list

# conda environments:
#
base                     /opt/conda



# Example of creating a simple receptor and a simple ligand, then moving the ligand out of the box

from: https://github.com/ccsb-scripps/AutoDock-Vina/issues/112

In [3]:
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from meeko import MoleculePreparation
from vina import Vina
import numpy as np
import vina

def print_ligand_center(molsetup):
    lig_xyz = []
    for atom_index, is_atom_ignored in molsetup.atom_ignore.items():
        if not is_atom_ignored:
            lig_xyz.append(molsetup.coord[atom_index].copy())
    lig_xyz = np.array(lig_xyz)
    print("ligand center: %8.3f %8.3f %8.3f" % tuple(np.mean(lig_xyz, 0)))

try:
    print("vina version:", vina.__version__)
except AttributeError:
    print("vina.__version__ not available:")

receptor_string = "ATOM      1  C   UNL     1       0.000   0.000   0.000  0.00  0.00    +0.000 C "
rec_fn = "my_rec_is_just_a_carbon.pdbqt"
print(receptor_string, file=open(rec_fn, "w"))

maps_center = [0, 0, 0]
maps_size_angstroms = [10, 10, 10]

pyridine = Chem.MolFromSmiles("C1=CC=CN=C1")
pyridine = Chem.AddHs(pyridine)
etkdg_params = rdDistGeom.ETKDGv3()
rdDistGeom.EmbedMolecule(pyridine, etkdg_params)

mkprep = MoleculePreparation()
mkprep.prepare(pyridine)
lig_string = mkprep.write_pdbqt_string()

v = Vina()
v.set_receptor(rec_fn)
v.set_ligand_from_string(lig_string)
v.compute_vina_maps(center=maps_center, box_size=maps_size_angstroms)

# ---------

print_ligand_center(mkprep.setup)
score = v.score()
print("score:", score)

if False: # this stops the kernel without even throwing an exception
    # translate ligand out of box
    for atom_index in mkprep.setup.coord:
        x, y, z = mkprep.setup.coord[atom_index]
        mkprep.setup.coord[atom_index] = np.array([x + 200, y + 200, z + 200])

    print_ligand_center(mkprep.setup)
    lig_string = mkprep.write_pdbqt_string()
    v.set_ligand_from_string(lig_string)
    score = v.score()
    print("score:", score)

vina version: 1.2.3
Computing Vina grid ... done.
ligand center:    0.034   -0.212    0.000
score: [29. 29.  0.  0.  0.  0.  0.  0.]


# One example using the vina first official example and try to move the protein from their original location

In [2]:
!wget https://raw.githubusercontent.com/ccsb-scripps/AutoDock-Vina/develop/example/python_scripting/1iep_ligand.pdbqt
!wget https://raw.githubusercontent.com/ccsb-scripps/AutoDock-Vina/develop/example/python_scripting/1iep_receptor.pdbqt

--2023-04-19 13:50:29--  https://raw.githubusercontent.com/ccsb-scripps/AutoDock-Vina/develop/example/python_scripting/1iep_receptor.pdbqt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216188 (211K) [text/plain]
Saving to: ‘1iep_receptor.pdbqt’

1iep_receptor.pdbqt 100%[===================>] 211.12K  --.-KB/s    in 0.01s   

2023-04-19 13:50:29 (17.1 MB/s) - ‘1iep_receptor.pdbqt’ saved [216188/216188]



In [3]:
from vina import Vina

# class vina.vina.Vina(sf_name='vina', cpu=0, seed=0, no_refine=False, verbosity=1)
v = Vina(sf_name='vina', seed=1)

v.set_receptor('1iep_receptor.pdbqt')

v.set_ligand_from_file('1iep_ligand.pdbqt')
v.compute_vina_maps(center=[15.190, 53.903, 16.917], box_size=[20, 20, 20])

# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# not in the original example
# v.write_pose('1iep_ligand_before_minimized.pdbqt', overwrite=True)

Computing Vina grid ... Score before minimization: -12.513 (kcal/mol)
done.


In [6]:
!wget https://raw.githubusercontent.com/ccsb-scripps/AutoDock-Vina/develop/example/basic_docking/data/1iep_ligand.sdf

--2023-04-19 17:23:02--  https://raw.githubusercontent.com/ccsb-scripps/AutoDock-Vina/develop/example/basic_docking/data/1iep_ligand.sdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6621 (6.5K) [text/plain]
Saving to: ‘1iep_ligand.sdf’

1iep_ligand.sdf     100%[===================>]   6.47K  --.-KB/s    in 0s      

2023-04-19 17:23:02 (45.1 MB/s) - ‘1iep_ligand.sdf’ saved [6621/6621]



In [ ]:
#from: https://pypi.org/project/meeko/0.3.3/

from meeko import MoleculePreparation
from rdkit import Chem

# input_molecule_file = 'example/BACE_macrocycle/BACE_4.mol2'
# mol = Chem.MolFromMol2File(input_molecule_file)
mol = Chem.SDMolSupplier('1iep_ligand.sdf')[0]

preparator = MoleculePreparation(hydrate=True) # macrocycles flexible by default since v0.3.0
preparator.prepare(mol)
preparator.show_setup()

# output_pdbqt_file = "test_macrocycle_hydrate.pdbqt"
# preparator.write_pdbqt_file(output_pdbqt_file)


In [15]:
%%file test_script_02.py

from vina import Vina
import numpy as np
from meeko import MoleculePreparation
from rdkit import Chem
from rdkit.Chem import rdDistGeom

def print_ligand_center(molsetup):
    lig_xyz = []
    for atom_index, is_atom_ignored in molsetup.atom_ignore.items():
        if not is_atom_ignored:
            lig_xyz.append(molsetup.coord[atom_index].copy())
    lig_xyz = np.array(lig_xyz)
    print("ligand center: %8.3f %8.3f %8.3f" % tuple(np.mean(lig_xyz, 0)))
    
def translate_ligand(mkprep, new_center):
    # translate ligand to new center
    for atom_index in mkprep.setup.coord:
        x, y, z = mkprep.setup.coord[atom_index]
        mkprep.setup.coord[atom_index] = np.array([x + new_center[0], y + new_center[1], z + new_center[2]])

# class vina.vina.Vina(sf_name='vina', cpu=0, seed=0, no_refine=False, verbosity=1)
v = Vina(sf_name='vina', seed=1)

v.set_receptor('1iep_receptor.pdbqt')

# pyridine = Chem.MolFromSmiles("C1=CC=CN=C1")
# pyridine = Chem.AddHs(pyridine)
# etkdg_params = rdDistGeom.ETKDGv3()
# rdDistGeom.EmbedMolecule(pyridine, etkdg_params)

mol = Chem.SDMolSupplier('1iep_ligand.sdf')[0]
mol = Chem.AddHs(mol)
etkdg_params = rdDistGeom.ETKDGv3()
rdDistGeom.EmbedMolecule(mol, etkdg_params)

mkprep = MoleculePreparation()
mkprep.prepare(mol)
# preparator.show_setup()
lig_string = mkprep.write_pdbqt_string()

v.set_ligand_from_string(lig_string)
maps_center=[15.190, 53.903, 16.917]
maps_size_angstroms=[20, 20, 20]
v.compute_vina_maps(center=maps_center, box_size=maps_size_angstroms)

# ---------

print_ligand_center(mkprep.setup)
translate_ligand(mkprep, maps_center)
print_ligand_center(mkprep.setup)

lig_string = mkprep.write_pdbqt_string()
v.set_ligand_from_string(lig_string)

score = v.score()
print("score:", score)


Overwriting test_script_02.py


In [16]:
!python3 test_script_02.py

Computing Vina grid ... done.
ligand center:    0.835   -0.234   -0.190
ligand center:   16.025   53.669   16.727
score: [ 2.00142e+02  2.82044e+02  0.00000e+00  0.00000e+00  0.00000e+00
 -4.10000e-02 -8.19020e+01 -4.10000e-02]


In [7]:
v.info()

{'rigid_receptor': '1iep_receptor.pdbqt',
 'flex_receptor': None,
 'ligands': ['1iep_ligand.pdbqt'],
 'scoring_function': 'vina',
 'weights': (-0.035579,
  -0.005156,
  0.840245,
  -0.035069,
  -0.587439,
  50,
  0.05846),
 'no_refine': False,
 'box_center': [15.19, 53.903, 16.917],
 'box_size': [20, 20, 20],
 'box_spacing': 0.375,
 'box_even_elements': True,
 'seed': 1}

In [10]:
# Minimized locally the current pose
energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
v.write_pose('1iep_ligand_minimized.pdbqt', overwrite=True)

Score after minimization : -13.206 (kcal/mol)
Performing local search ... done.


In [11]:
# # Dock the ligand
v.dock(exhaustiveness=32, n_poses=20)
v.write_poses('1iep_ligand_vina_out.pdbqt', n_poses=5, overwrite=True)

Performing docking (random seed: 1) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
